# Notebook initialization
- Importing libraries
- Configuration - setting up parameters

In [0]:
import json
import requests
from azure.identity import ClientSecretCredential

In [0]:
tenant_id = 'your-tenant-id-from-azuread'

#key-vault-dbxsecrets - this is databricks secret scope, that points to Azure Active Directory Key Vault
client_id = dbutils.secrets.get(scope = 'key-vault-dbxsecrets', key = 'your-sp-appid')
client_secret = dbutils.secrets.get(scope = 'key-vault-dbxsecrets', key = 'your-sp-secret')

server_name = 'your-azure-sql-server-name'
db_name = 'your-azure-database-name'

db_scope = 'https://database.windows.net/.default'
azure_sql_url = f'jdbc:sqlserver://{server_name}.database.windows.net'
database = f'{db_name}'

## Requesting token

In [0]:
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                                                        tenant_id = tenant_id,
                                                        client_id = client_id,
                                                        client_secret = client_secret)

db_access_token = auth.get_token(db_scope)
db_access_token = db_access_token.token

## Some useful functions

In [0]:
def read_table(table_name: str):
    return spark.read.format('jdbc').option('url', azure_sql_url).option('dbtable', table_name).option('database', database).option('accessToken', db_access_token).load()

In [0]:
def execute_query(query: str):
    return spark.read.format('jdbc').option('url', azure_sql_url).option('query', query).option('database', database).option('accessToken', db_access_token).load()

In [0]:
def write_table(df, table_name:str):    
    return df.write.format('jdbc').option('url', azure_sql_url).option('dbtable', table_name).option('database', database).option('accessToken', db_access_token).mode('append').save()

In [0]:
def truncate_and_write_table(df,table_name: str):
    return df.write.format('jdbc').option('url', azure_sql_url).option('truncate', 'true').option('dbtable', table_name).option('database', database).option('accessToken', db_access_token).mode('overwrite').save()